In [4]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, ConvLSTM2D, Conv2D, BatchNormalization

def create_convlstm_model(input_shape):
    inputs = Input(shape=input_shape)

    x = ConvLSTM2D(filters=64, kernel_size=(3, 3), padding="same", return_sequences=True)(inputs)
    x = BatchNormalization()(x)

    x = ConvLSTM2D(filters=64, kernel_size=(3, 3), padding="same", return_sequences=True)(x)
    x = BatchNormalization()(x)

    x = ConvLSTM2D(filters=32, kernel_size=(3, 3), padding="same", return_sequences=True)(x)
    x = BatchNormalization()(x)

    x = ConvLSTM2D(filters=32, kernel_size=(3, 3), padding="same", return_sequences=False)(x)
    x = BatchNormalization()(x)

    outputs = Conv2D(filters=3, kernel_size=(3, 3), activation='sigmoid', padding="same")(x)

    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Load and preprocess frames
def load_video_frames(video_path, frame_shape, seq_length):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, frame_shape[:2])
        frame = frame / 255.0  # Normalize to [0,1]
        frames.append(frame)

    cap.release()

    # Prepare input-output pairs: each sequence has `seq_length` frames, and the target is the next frame
    sequences, targets = [], []
    for i in range(len(frames) - seq_length):
        sequences.append(frames[i:i + seq_length])
        targets.append(frames[i + seq_length])  # The next frame after the sequence

    return np.array(sequences), np.array(targets)

# Parameters
video_path = "input_video.mp4"
frame_shape = (128, 128, 3)
seq_length = 10  # Number of frames in each sequence

# Load frames and create input-output pairs for training
train_sequences, train_targets = load_video_frames(video_path, frame_shape, seq_length)

# Define the input shape for the ConvLSTM model
input_shape = (seq_length, frame_shape[0], frame_shape[1], frame_shape[2])

# Create ConvLSTM model
model = create_convlstm_model(input_shape)

# Train the model
model.fit(train_sequences, train_targets, epochs=10, batch_size=2)

# Generate next 25 frames and save them
def generate_next_frames(model, initial_sequence, num_frames, save_dir):
    generated_frames = []
    current_sequence = initial_sequence.copy()

    for i in range(num_frames):
        # Predict the next frame
        next_frame = model.predict(np.expand_dims(current_sequence, axis=0))[0]
        generated_frames.append(next_frame)

        # Save frame to the specified directory
        frame_path = os.path.join(save_dir, f"generated_frame_{i+1}.png")
        cv2.imwrite(frame_path, (next_frame * 255).astype(np.uint8))

        # Update the sequence by removing the first frame and appending the new frame
        current_sequence = np.concatenate((current_sequence[1:], np.expand_dims(next_frame, axis=0)))

    return generated_frames

# Use the last sequence from the video to generate future frames
initial_sequence = train_sequences[-1]

# Define directory to save frames
save_dir = os.path.dirname(video_path)
generated_frames = generate_next_frames(model, initial_sequence, num_frames=25, save_dir=save_dir)

print("Generated frames have been saved to:", save_dir)


Epoch 1/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 52s 459ms/step - loss: 0.0310
Epoch 2/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 73s 411ms/step - loss: 0.0061
Epoch 3/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 35s 407ms/step - loss: 0.0053
Epoch 4/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 35s 406ms/step - loss: 0.0045
Epoch 5/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 41s 409ms/step - loss: 0.0045
Epoch 6/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 35s 407ms/step - loss: 0.0035
Epoch 7/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 35s 407ms/step - loss: 0.0044
Epoch 8/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 35s 407ms/step - loss: 0.0033
Epoch 9/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 41s 407ms/step - loss: 0.0033
Epoch 10/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 41s 407ms/step - loss: 0.0036
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━